# Injecting small scales to Spectral index and dust Temperature templates


- We inject smaller angular scales to the maps  by extrapolating the power law fitted from the GNILC spectral parameter maps 

- Smaller angular scales are modulated similarly as the intensity map in pol tens formalism. 

- the multipoles where the fit is performed are different given the observed spectra . In any case we don't fit beyond $\ell=400$, which is consistent with the TT analysis 
- given the fact that we inject smaller angular scales with a steeper spectral index  than TT
$$\gamma_{\beta} = -1.96, \gamma_{Td} = -2.47, \gamma_{TT}= -1.29$$

we don't expect to inject _small scale noise_ when rescaling at frequencies orders of magnitude lower or larger than  the reference one (353 GHz).


In [ ]:
import os

os.environ[
    "OMP_NUM_THREADS"
] = "64"  # for jupyter.nersc.gov otherwise the notebook only uses 2 cores

In [ ]:
from pathlib import Path
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np

# import pymaster as nmt
from astropy.io import fits

%matplotlib inline

In [ ]:
hp.disable_warnings()

In [ ]:
plt.style.use("seaborn-talk")

In [ ]:
import pysm3 as pysm
import pysm3.units as u

In [ ]:
nside = 2048
lmax = 2048

In [ ]:
comp = "IQU"

In [ ]:
components = list(enumerate(comp))
components

In [ ]:
spectra_components = ["TT", "EE", "BB"]

In [ ]:
datadir = Path("data")

## Modulation of small scales

Small scales are generated uniform across the sky, then modulated with a factor based on the Dust intensity map.

The modulation factor is scaled differently in the high emission and in the low emission parts of the map.

In [ ]:
logm = hp.read_map(datadir / "dust_gnilc_logpoltens_varres_nomono.fits")
ismooth = hp.smoothing(logm, fwhm=np.radians(5), lmax=lmax)

In [ ]:
b1 = 1.1
b2 = 1.5
a = 0.1
minmax = lambda m, a, b: a + (b - a) * (m - m.min()) / (m.max() - m.min())

modulate_amp = (ismooth) * 1.0
val = 4.5
mskmd = ismooth > val

modulate_amp[mskmd] = minmax(ismooth[mskmd], a=b1, b=b2)

modulate_amp[~mskmd] = minmax(ismooth[~mskmd], a=a, b=b1)

In [ ]:
hp.mollview(mskmd, title="High emission/low emission mask")

In [ ]:
hp.mollview(modulate_amp, title="Modulation amplitude")

In [ ]:
hp.write_map(datadir / "gnilc_dust_beta_Tdust_modulation_map.fits", modulate_amp, coord="G", column_units="dimensionless", overwrite=True, dtype=np.float32)

In [ ]:
fits.open(datadir / "gnilc_dust_beta_Tdust_modulation_map.fits")[1].header

In [ ]:
alm_modulate_amp = hp.map2alm(modulate_amp, lmax=1.5 * nside, use_pixel_weights=True)

In [ ]:
hp.write_alm(datadir / f"gnilc_dust_beta_T_dust_modulation_alms_lmax{1.5*nside}.fits", alm_modulate_amp, overwrite=True, out_dtype=np.float32)

In [ ]:
del modulate_amp

## Read and fit beta and T dust

In [ ]:
tdfile = datadir / "COM_CompMap_Dust-GNILC-Model-Temperature_2048_R2.00.fits"
if not tdfile.exists():
    !wget -O $tdfile  http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CompMap_Dust-GNILC-Model-Temperature_2048_R2.00.fits

bdfile = datadir / "COM_CompMap_Dust-GNILC-Model-Spectral-Index_2048_R2.00.fits"
if not bdfile.exists():
    !wget -O $bdfile http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=COM_CompMap_Dust-GNILC-Model-Spectral-Index_2048_R2.00.fits

In [ ]:
td = hp.read_map(tdfile)
bd = hp.read_map(bdfile)

In [ ]:
cltd = hp.anafast(td, lmax=lmax)
clbd = hp.anafast(bd, lmax=lmax)

cl = {"bd": clbd, "td": cltd}
dust_params = list(cl.keys())
ell = np.arange(lmax + 1)

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def model(ell, A, gamma):
    out = A * ell ** gamma
    out[:2] = 0
    return out

In [ ]:
ell_fit_low = {"bd": 200, "td": 100}
ell_fit_high = {"bd": 400, "td": 400}

A_fit, gamma_fit, A_fit_std, gamma_fit_std = {}, {}, {}, {}
plt.figure(figsize=(25, 5))

for ii, pol in enumerate(dust_params):
    plt.subplot(131 + ii)
    xdata = np.arange(ell_fit_low[pol], ell_fit_high[pol])
    ydata = cl[pol][xdata]
    (A_fit[pol], gamma_fit[pol]), cov = curve_fit(model, xdata, ydata)

    A_fit_std[pol], gamma_fit_std[pol] = np.sqrt(np.diag(cov))
    plt.loglog(ell, cl[pol], label=" Anafast $C_\ell$")

    plt.plot(
        ell[ell_fit_low[pol] // 2 : ell_fit_high[pol] * 2],
        model(
            ell[ell_fit_low[pol] // 2 : ell_fit_high[pol] * 2],
            A_fit[pol],
            gamma_fit[pol],
        ),
        label="model fit",
    )

    plt.axvline(
        ell_fit_low[pol],
        linestyle="--",
        color="black",
        label="$ \ell={} $".format(ell_fit_low[pol]),
    )
    plt.axvline(
        ell_fit_high[pol],
        linestyle="--",
        color="gray",
        label="$ \ell={} $".format(ell_fit_high[pol]),
    )
    plt.legend()
    plt.grid()
    plt.title(
        f"{pol}   spectrum for dust   " + r"$\gamma_{fit}=$" + f"{gamma_fit[pol]:.2f}"
    )

    plt.ylabel("$ C_\ell $")
    plt.xlabel(("$\ell$"))
    plt.xlim(2, lmax)

## Preprocess large scales

In [ ]:
def sigmoid(x, x0, width, power=4):
    """Sigmoid function given start point and width
    Parameters
    ----------
    x : array
        input x axis
    x0 : float
        value of x where the sigmoid starts (not the center)
    width : float
        width of the transition region in unit of x
    power : float
        tweak the steepness of the curve
    Returns
    -------
    sigmoid : array
        sigmoid, same length of x"""
    return 1.0 / (1 + np.exp(-power * (x - x0 - width / 2) / width))

In [ ]:
output_lmax = 8192*2
output_ell = np.arange(output_lmax + 1)

In [ ]:
np.nonzero(np.sqrt(1-sig_func) < 1e-10)[0][0]

In [ ]:
output_lmax_LS = 1024

In [ ]:
alm_bd = hp.map2alm(bd, lmax=lmax, use_pixel_weights=True)
alm_td = hp.map2alm(td, lmax=lmax, use_pixel_weights=True)

bd_LS_alm = np.empty_like(alm_bd)
td_LS_alm = np.empty_like(alm_bd)
sig_func = sigmoid(ell, x0=ell_fit_high["bd"], width=ell_fit_high["bd"] / 10)
bd_LS_alm = hp.almxfl(alm_bd, np.sqrt(1.0 - sig_func))
td_LS_alm = hp.almxfl(alm_td, np.sqrt(1.0 - sig_func))

In [ ]:
hp.write_alm(
    datadir / f"gnilc_largescale_beta_alm_lmax{output_lmax_LS}_complex64.fits",
    bd_LS_alm,
    out_dtype=np.complex64,
    lmax=output_lmax_LS,
    overwrite=True,
)
hp.write_alm(
    datadir / f"gnilc_largescale_Tdust_alm_lmax{output_lmax_LS}_complex64.fits",
    td_LS_alm,
    out_dtype=np.complex64,
    lmax=output_lmax_LS,
    overwrite=True,
)

## Generate small scales

In [ ]:
np.random.seed(777)
# filter small scales
small_scales_input_cl = [
    1
    * model(output_ell, A_fit[pol], gamma_fit[pol])
    * (sigmoid(output_ell, ell_fit_high[pol], ell_fit_high[pol] / 10))
    for pol in dust_params
]

In [ ]:
hp.write_cl(
    datadir / f"gnilc_dust_beta_Tdust_small_scales_cl_lmax{output_lmax}.fits",
    small_scales_input_cl,
    dtype=np.complex128,
    overwrite=True,
)

In [ ]:
np.random.seed(777)
bd_ss_alm = hp.synalm(small_scales_input_cl[0], lmax=output_lmax)
np.random.seed(555)
td_ss_alm = hp.synalm(small_scales_input_cl[1], lmax=output_lmax)

## Combine all scales to output map

In [ ]:
bd_ls = hp.alm2map(bd_LS_alm, nside=output_nside)
td_ls = hp.alm2map(td_LS_alm, nside=output_nside)

In [ ]:
modulate_amp = alm2map(alm_modulate_amp, nside=output_nside)

In [ ]:
bd_ss = hp.alm2map(bd_ss_alm, nside=output_nside)
td_ss = hp.alm2map(td_ss_alm, nside=output_nside)

bd_ss *= modulate_amp
td_ss *= modulate_amp

bd_out = bd_ss + bd_ls
td_out = td_ss + td_ls

In [ ]:
hp.write_map(datadir / f"gnilc_dust_beta_map_nside{output_nside}.fits", bd_out, coord="G", column_units="dimensionless", overwrite=True, dtype=np.float32)

In [ ]:
hp.write_map(datadir / f"gnilc_dust_Tdust_map_nside{output_nside}.fits", td_out, coord="G", column_units="K", overwrite=True, dtype=np.float32)

In [ ]:
cl_out = {"bd": hp.anafast(bd_out, use_pixel_weights=True, lmax=output_lmax), "td": hp.anafast(td_out, use_pixel_weights=True, lmax=output_lmax)}

In [ ]:
for ii, pol in enumerate(dust_params):
    plt.loglog(ell, cl[pol], alpha=0.5, color="C%d" % ii)
    plt.loglog(ell, cl_out[pol], label=f" {pol}   ", color="C%d" % ii)

    plt.legend()
    plt.grid(True)
    plt.plot(
        ell[100:][2:],
        model(ell[100:], A_fit[pol], gamma_fit[pol])[2:],
        "--",
        color="red",
    )
    plt.axvline(ell_fit_high[pol], linestyle="--", color="gray")
    plt.axvline(100, linestyle="--", color="gray")

    plt.ylabel("$ C_\ell $")
    plt.xlabel(("$\ell$"))
    # plt.xlim(350, 500 )
    plt.xlim(2, lmax)

## Compare input and outputs on a small patch of sky

In [ ]:
m_planck_varres = hp.read_map(
    datadir / "COM_CompMap_Dust-GNILC-F353_2048_21p8acm.fits", dtype=np.float64
)

In [ ]:
lat = 35
cm = plt.cm.RdBu
plt.figure(figsize=(15, 8))
hp.gnomview(
    bd_out,
    title="Bd  w/ small scales ",
    rot=[0, lat],
    reso=3.75,
    cmap=cm,
    xsize=320,
    sub=234,
    min=1.2,
    max=1.9,
)
hp.gnomview(
    bd_ss, title="small scales ", rot=[0, lat], reso=3.75, xsize=320, cmap=cm, sub=232
)
hp.gnomview(
    m_planck_varres,
    title=" GNILC I MAP  ",
    rot=[0, lat],
    reso=3.75,
    cmap=cm,
    xsize=320,
    sub=233,
)

hp.gnomview(
    (bd),
    title=" Bd  GNILC  ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=235,
    min=1.2,
    max=1.9,
)
hp.gnomview(
    bd_ls,
    title="  Bd large scales ",
    rot=[0, lat],
    reso=3.75,
    cmap=cm,
    xsize=320,
    sub=231,
    min=1.2,
    max=1.9,
)

plt.figure(figsize=(15, 8))
hp.gnomview(
    td_out,
    title="Td  w/ small scales ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=234,
    min=15,
    max=25,
)
hp.gnomview(
    td_ss, title="small scales ", rot=[0, lat], reso=3.75, xsize=320, sub=232, cmap=cm
)
hp.gnomview(
    (modulate_amp),
    title=" modulation  ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=233,
)

hp.gnomview(
    (td),
    title=" Td  GNILC  ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    sub=235,
    cmap=cm,
    min=15,
    max=25,
)
hp.gnomview(
    td_ls,
    title="  Td large scales ",
    rot=[0, lat],
    reso=3.75,
    xsize=320,
    cmap=cm,
    sub=231,
    min=15,
    max=25,
)